## 1. Importing the necessary libraries and defining constants.

In [174]:
import polars as pl
import polars.selectors as sc

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.colors as colors

import pathlib as path
import shutil 
 # Define the ratio of the plot.
fig_aspect_ratio : float = 4/3 # width/height ratio
height : float = 6
width: float = height * fig_aspect_ratio


# Assign an ordinal number to each month to promote the proper order of the months in directories (January, February etc.)
month_order : dict[str, int] = {
                "January":1,
                "February":2,
                "March":3,
                "April":4,
                "May":5,
                "June":6,
                "July":7,
                "August":8,
                "September":9,
                "October":10,
                "November":11,
                "December":12
                }

            
main_path: path.Path = path.Path().cwd()/"Financial summaries"

if main_path.exists():
    shutil.rmtree(main_path)



# 2. Reading the data from Excel sheets.

In [175]:
expenses_df: pl.DataFrame = pl.read_excel("Expenses.xlsx", 
                                       sheet_name = "Expenses",
                                       has_header = True)

income_df:pl.DataFrame = pl.read_excel("Incomes.xlsx", 
                                       sheet_name = "Incomes",
                                       has_header = True)

expenses_df

Expense date,Where bought?,What bought?,Expense cost,Volume unit,Volume amount,Expense category,Product category
date,str,str,f64,str,f64,str,str
2024-02-01,"""Biedronka""","""Ser żółty światowid""",13.98,"""gram""",600.0,"""Groceries""","""Ser"""
2024-02-01,"""Biedronka""","""Woda gazowana muszynianka""",2.99,"""ml""",1500.0,"""Drinks""","""Woda"""
2024-02-01,"""Biedronka""","""Ryż brązowy plony natury""",5.98,"""gram""",800.0,"""Groceries""","""Ryż"""
2024-02-01,"""Biedronka""","""Serek wiejski""",5.97,"""gram""",600.0,"""Groceries""","""Serek wiejski"""
2024-02-01,"""Biedronka""","""Olej słonecznikowy""",7.99,"""ml""",1000.0,"""Groceries""","""Olej spożywczy"""
…,…,…,…,…,…,…,…
2025-02-20,"""Pizzeria Diabolo""","""Sos czosnkowy""",2.5,"""sztuka""",1.0,"""DiningOut""","""Sos"""
2025-02-21,"""Rossmann""","""Krem do rąk Neutrogena""",11.99,"""ml""",75.0,"""Hygiene""","""Krem do rąk"""
2025-02-21,"""Hubert Górski""","""Tankowanie samochodu""",26.0,"""sztuka""",1.0,"""Travel""","""Gas/benzyna"""


# 3. Grouping the costs by month and year.

In [176]:
grouped_expense: pl.DataFrame = (   expenses_df.group_by(pl.col("Expense date").dt.year().alias("Year"),
                                                   pl.col("Expense date").dt.quarter().alias("Quarter"),
                                                   pl.col("Expense date").dt.strftime("%b").alias("Month"),
                                                   maintain_order = True).
                                                   agg(pl.col("Expense cost").sum().alias("Total expenses")) )


grouped_income:pl.DataFrame = ( income_df.group_by( pl.col("Income date").dt.year().alias("Year"),
                                                 pl.col("Income date").dt.quarter().alias("Quarter"), 
                                                 pl.col("Income date").dt.strftime("%b").alias("Month"), 
                                                 maintain_order = True).
                                                agg(pl.col("Income amount").sum().alias("Total incomes")) )


## 4. Preprocessing the balance sheets for plotting.

In [177]:
# Join the grouped_income with grouped_cost.
balances:pl.DataFrame    = grouped_income.join(grouped_expense, on = (pl.col("Year"), pl.col("Month")),
                                                    how = "full").select(~sc.ends_with("right")).fill_null(0)


# Add a column "Miesięczne saldo"
balances:pl.DataFrame = balances.with_columns( (pl.col("Total incomes") - pl.col("Total expenses")).alias("Monthly balance"))


# Convert the balance sheet to long form.
balances_long:pl.DataFrame = balances.unpivot(index = ("Year","Quarter", "Month"),
                                                              variable_name = "Financial category",
                                                             value_name = "Amount")

fin_cat:pl.Expr = pl.col("Financial category")


hue_variable: pl.Expr = ( pl.when( fin_cat.is_in( ["Total incomes", "Total expenses"])).
                         then(
                             fin_cat
                             ).otherwise( 
                                 pl.when(pl.col("Amount") >=0).
                                 then(pl.lit("Positive balance")).
                                 otherwise(pl.lit("Negative balance"))
                             ).alias("Financial category") 
                             )

balances_long: pl.DataFrame = balances_long.with_columns(hue_variable)

balances_long


Year,Quarter,Month,Financial category,Amount
i32,i8,str,str,f64
2024,1,"""Feb""","""Total incomes""",1468.0
2024,1,"""Mar""","""Total incomes""",1070.0
2024,2,"""Apr""","""Total incomes""",1606.5
2024,2,"""May""","""Total incomes""",1630.0
2024,2,"""Jun""","""Total incomes""",1928.0
…,…,…,…,…
2024,4,"""Oct""","""Positive balance""",448.86
2024,4,"""Nov""","""Positive balance""",515.225
2024,4,"""Dec""","""Positive balance""",2380.78


## 5. Visualizing the annual general balance sheets.

In [178]:
monthly_balances:pd.DataFrame = balances_long.filter(pl.col("Financial category").str.ends_with("balance"))

# Create a Normalizer for our data for the color map to work properly.
Normalize = colors.Normalize(monthly_balances["Amount"].min(), monthly_balances["Amount"].max())

# Find the "turning" normalized value (that is, if Normalize(x) < x0 then x is negative, otherwise - positive)
x0 = Normalize(0)

# Define the color entries for the LinearSegmentedColorMap.
red : list[tuple[int]] = [(0, 0, 0.5), (x0, 1, 0), (1, 0, 0)]
green : list[tuple[int]] = [(0, 0, 0), (x0, 0 ,0.5), (1, 1, 1)]
blue : list[tuple[int]] =  [(0, 0, 0), (1, 0, 0)]
alpha: list[tuple[int]] = [(0, 1, 1), (x0, 0, 0), (1, 1,1)]



# Let's create a custom color map
financial_colormap = colors.LinearSegmentedColormap("FinancialPalette", 
                                            {"red": red, "green": green, "blue": blue,
                                            "alpha": alpha})



def generate_month_balance_sheet(year:int) -> None:
    """The function generates the month balance sheet plot for a given year. All of the plots are saved in a given path.
    
    Parameters:
    ---------
    year: int
        The year of the month balances.

    Returns:
    None

    """
    # 



    # Filter the balance sheet by year.
    df_filtered = monthly_balances.filter(pl.col("Year") == year)


    # Convert the values to color.
    kolory = [financial_colormap(Normalize(v)) for v in df_filtered["Amount"]] 


    # Create a figure
    global width, height
    figure = plt.figure(figsize = (width, height))
    axes = sns.barplot(data=df_filtered, x="Month", y="Amount", 
                       palette=kolory, hue = "Month", 
                       legend = False,
                       edgecolor = "black", linewidth = 1.5)


    xlim: tuple[float] = axes.get_xlim()

    # Customizing the plot.
    axes.spines[["top", "right"]].set_visible(False)
    axes.grid(True, alpha = 0.2)

    # Setting the title of the plot and axis labels.
    axes.set_title("Monthly balances in \n"+ rf"$\bf{{\ {year}}}$")
    axes.set_ylabel("Monthly balance")
    axes.set_xlabel("Balance month")

    # Adjust the  bounds of the bottom spine
    xlim: tuple[float] = axes.get_xlim()

    # Rebounding the bottom spine.
    axes.spines["bottom"].set_bounds(xlim[0], xlim[1])
    
    # Set the position of the left spine.
    axes.spines["left"].set_position(("data",xlim[0]))

    # Add a new Ox axis at position y =0.
    axes.hlines(0, xlim[0], xlim[1], colors = "black")
    
    # Set the major formatter for the yaxis.
    axes.yaxis.set_major_formatter(lambda x,v: f"{x:,.0f} zł".replace(",", " "))

    
    for container in axes.containers:
        axes.bar_label(container, fmt = lambda v: f"{v:,.0f} zł".replace(",", " "), padding = 2)


    saving_path  = main_path/rf"{year}"

    if not saving_path.exists():
        saving_path.mkdir(parents = True)
    
    figure.savefig(fname = saving_path/r"General monthly balance.png")
    plt.close()


# Generate the month balance sheets for each year available in the dataset.
for year in monthly_balances["Year"].unique():
    generate_month_balance_sheet(year)



# 7. Visualizing the income and expenses for each month.

### For a given year, we group the income and expenses by quarter and visualize these groups on a single plot.

In [179]:
monthly_detailed_balances_path: path.Path = path.Path().cwd()/r"Detailed monthly balances"


def generate_month_detailed_balances(year:int) -> None:
    # Filter the balance sheets and add a new column for "signed" amount.
    year_balance_sheets: pl.DataFrame = ( balances_long.
                                        filter(pl.col("Year") == year, 
                                                pl.col("Financial category").str.starts_with("Total"))
                                                
                                                )


    FacetGrid = sns.FacetGrid(data = year_balance_sheets, col = "Quarter", col_wrap = min(2, year_balance_sheets["Quarter"].n_unique()), sharex = False, sharey = False)
    # Set title for each subplot.
    FacetGrid.set_titles(template = "Monthly balances in {col_name}th quarter")
    # Set the global title for the whole figure.

    FacetGrid.fig.suptitle(fr"Detailed monthly balances for  $\bf{{{year}}}$",
                           size = 15)
    
    FacetGrid.fig.subplots_adjust(top=0.9)  # Lif the title a little bit.



    for quarter_id, axes in FacetGrid.axes_dict.items():
        # Find the rows from a given quarter.
        quarter_balance_sheets: pl.DataFrame = year_balance_sheets.filter(pl.col("Quarter") == quarter_id)

        sns.barplot(quarter_balance_sheets, x = "Month", y = "Amount", hue = "Financial category",
                    ax = axes,
                    palette = {"Total expenses":"red", "Total incomes":"green"},
                    edgecolor = "black",
                    linewidth = 1.75
                    )

        # Find the minimum and maximum amount respectively.
        y_min, y_max = quarter_balance_sheets["Amount"].min(), quarter_balance_sheets["Amount"].max()

        axes.get_legend().set_title("")

        axes.set_xlabel("Balance month", labelpad = -1)

        axes.set_ylim(0, y_max*(  1.1 + (y_max-y_min)/y_max))
        axes.yaxis.set_major_formatter(lambda x,v: f"{x:,.0f} zł".replace(",", " "))

        axes.grid(True, alpha = 0.3)

        # Add bar labels.
        for i in range(len(axes.containers)):
            axes.bar_label(axes.containers[i], fmt = lambda v: f"{v:,.0f}".replace(",", " "),
                           padding = 2)
        
        # Resize the width of the bars so that the bar labels wouldn't overlap.
        q: float = 0.7
        for patch in axes.patches:
           current_width: float = patch.get_width()

           patch.set_width(w = current_width*q)
           patch.set_x( patch.get_x() +  current_width/2 *(1-q))
        
    # Get the figure.
    figure: plt.Figure = axes.get_figure()
    
    # Change the shapes of the figure.
    figure.set_size_inches((8*fig_aspect_ratio, 8))

    saving_path = main_path/f"{year}"

    if not saving_path.exists():
        saving_path.mkdir(parents = True)
    
    figure.savefig(saving_path/rf"Detailed monthly balance")
    plt.close()

  


for year in balances_long["Year"].unique():
    generate_month_detailed_balances(year)  

# 8. Visualizing the share of expense categories in monthly spendings.

In [180]:
import numpy as np

def plot_the_share(df:pl.DataFrame, 
                   date_col:str, cat_col:str, cost_col: str, 
                   transaction_type:str,
                     ) -> None:
    """"The function visualize the categories share in either expenses or incomes.

    Parameters:
    df : pl.DataFrame
        The dataframe storing the expenses or incomes.

    date_col : str
        The name of the date column.

    cat_col : str
        The name of the (expense/income) category column.

    cost_col : str
        The name of the cost column.

    transaction_type : str
        One of two of types of transaction (expenses, costs) we're vizualising.

    
    """
    cat_share: pl.DataFrame = df.group_by(pl.col(date_col).dt.year().alias("Year"),
                                                            pl.col(date_col).dt.strftime("%B").alias("Month"),
                                                            pl.col(cat_col)).agg(pl.sum(cost_col).alias("Total amount"))

    for year in cat_share["Year"].unique():
        for month in cat_share["Month"].unique():
            # Filter out the monthly expenses and add a normalized amount of the expenses. 
            cat_share_filtered : pl.DataFrame = ( cat_share.filter(pl.col("Year")== year,pl.col("Month") == month).
            with_columns((100*pl.col("Total amount")/pl.sum("Total amount")).alias('Share'))
            )

        

            if cat_share_filtered.shape[0]:
                # Create a condition to group the least-expensive categories.
                grouping_condition: pl.Expr = ( pl.when(pl.col("Share") <=1.5)
                .then(pl.lit("Other"))
                .otherwise(pl.col(cat_col)).
                alias(cat_col) )

                # Group the least-expensive categories together.
                cat_share_filtered : pl.DataFrame = ( cat_share_filtered.with_columns(grouping_condition )
                .group_by(pl.col(cat_col)).
                agg(pl.sum("Total amount"), pl.sum("Share")) ).sort(pl.col("Share"), descending = True)


                # Create a figure.
                fig, ax = plt.subplots(figsize=(6, 3))

                # Resize the figure.
                global height, width
                fig.set_size_inches(width, height)


                wedges, empty_labels = plt.pie(cat_share_filtered["Share"],
                        wedgeprops = {"edgecolor":(0, 0,0, 0.75), "linewidth":1},
                        )
                
                # Find the labels of each wedge.
                labels : pl.Series = cat_share_filtered[cat_col]


                # Define the formatter for procentage share.
                text_formatter: callable = lambda t: f"{t:.2f}" 

                for label, wedge, share in zip(labels, wedges, cat_share_filtered["Share"]):

                    # Find the starting and ending angles of the wedge.
                    theta1, theta2 = wedge.theta1, wedge.theta2

                    # Find the radius of the wedge.
                    radius: float = wedge.r

                    # Find the angle of the point the label will be locacated at.
                    angle:float = np.deg2rad((theta1+2*theta2)/3)

            
                    # Find the cordinates of the circle edge point.
                    x:float = np.cos(angle) * radius
                    y:float = np.sin(angle) * radius


                    # The point from the circumference.
                    xy:np.ndarray = np.array([x,y])
                    
                    # The tail point scaling factor.
                    if x >= 0:
                        q:float = 1.2
                    else:
                        q: float = 1.2 if y >= 0 else 1.3


                    # The arrow tail cordinates caling factor, which should be in the [0;1] Interval
                    s:float = 0.9

                    # Cind the tail and head of the arrow.
                    arrow_tail_xy: float = s*xy
                    arrow_head_xy: float = q*xy


                    # Create a good-looking label depending on the orientation of the Wedge.
                    text = (label.replace(" ", "\n")  + (", " if x >=0 else  "\n")  + fr"$\bf{{{text_formatter(share)}}}\%$")


                    ax.annotate(text = text , xy = arrow_tail_xy, xytext = arrow_head_xy,
                                arrowprops = {"arrowstyle":"<-", "connectionstyle":"Angle3",
                                            "color":"black", "linewidth":1, "linestyle":"dashed"},
                                fontsize = 9
                                    
                                )   
                    
                ax.set_title(f"Monthly categories share in {transaction_type.lower()} in " +rf"$\bf{{{month}, {year}}}$",loc = "right",
                fontsize  = 12)

                #Save the figure.
                saving_path = main_path/f"{year}"/f"{month_order[month]} {month}"
                saving_path.mkdir(parents = True, exist_ok = True)

                #Save the figure.
                fig.savefig(fname = saving_path/f"Categores share in {transaction_type.lower()}")
                plt.close()

             
               

plot_the_share(expenses_df, "Expense date", "Expense category", "Expense cost", "Expenses")
plot_the_share(income_df, "Income date", "Income category", "Income amount", "Incomes")

# 9. Average monthly expenses

In [ ]:
# Group the expenses by date and expense category.
grouped_expenses = (
    expenses_df.group_by(
        pl.col("Expense date").alias("Date"),
        pl.col("Expense category").alias("Category")
    ).agg(
        pl.sum("Expense cost").alias("Cost")
    )
)

# Define grouping variables for year, month, and category.
grouping_vars = [
    pl.col("Date").dt.year().alias("Year"),
    pl.col("Date").dt.month().alias("Month"),
    pl.col("Category")
]

# Group the expenses by year, month, and category, then sum the costs.
df = grouped_expenses.group_by(grouping_vars).agg(pl.sum("Cost"))

# Group again by month and category, then compute the average cost and cast the average cost to integer.
df = df.group_by(
    pl.col("Month"),
    pl.col("Category")
).agg(
    pl.mean("Cost").cast(pl.Int32).alias("Avg cost")
)

# Sort the DataFrame.
df = df.sort(
    pl.col("Month"),
    pl.col("Category")
)

# Pivot the DataFrame to get the monthly average expenses.
avg_monthly_cost = df.pivot(
    on="Month",
    index="Category",
    values="Avg cost"
).fill_null(0)


# Group the expenses by year and category, then sum the costs.
annual_grouped_expenses = (
    grouped_expenses.group_by(
        pl.col("Date").dt.year().alias("Rok"),
        pl.col("Category")
    ).agg(
        pl.sum("Cost")
    )
)

# Group again by category and compute the average annual cost.
avg_annual_cost = (
    annual_grouped_expenses.group_by(
        pl.col("Category")
    ).agg(
        pl.mean("Cost").cast(pl.Int32).alias("Average annual cost")
    )
)



avg_monthly_cost = ( 
                avg_monthly_cost.join(avg_annual_cost, on="Category"
                ).sort(pl.col("Average annual cost"), 
                descending = True
                ) 
)



saving_path = main_path/"Average monthly expenses"
saving_path.mkdir(parents = True, exist_ok = True)

# Save the DataFrame to a CSV file.
avg_monthly_cost.write_csv(saving_path/"Average monthly expenses.csv",
                           include_header = True,
                           separator=" ",
)

shape: (23, 14)
┌─────────────┬──────┬──────┬──────┬───┬──────┬──────┬─────┬─────────────────────┐
│ Category    ┆ 1    ┆ 2    ┆ 3    ┆ … ┆ 10   ┆ 11   ┆ 12  ┆ Average annual cost │
│ ---         ┆ ---  ┆ ---  ┆ ---  ┆   ┆ ---  ┆ ---  ┆ --- ┆ ---                 │
│ str         ┆ i32  ┆ i32  ┆ i32  ┆   ┆ i32  ┆ i32  ┆ i32 ┆ i32                 │
╞═════════════╪══════╪══════╪══════╪═══╪══════╪══════╪═════╪═════════════════════╡
│ Housing     ┆ 1650 ┆ 1205 ┆ 1013 ┆ … ┆ 1250 ┆ 1270 ┆ 53  ┆ 7655                │
│ Groceries   ┆ 471  ┆ 301  ┆ 498  ┆ … ┆ 466  ┆ 340  ┆ 297 ┆ 2488                │
│ Recreation  ┆ 50   ┆ 113  ┆ 2008 ┆ … ┆ 0    ┆ 34   ┆ 119 ┆ 1305                │
│ Electronics ┆ 0    ┆ 0    ┆ 47   ┆ … ┆ 0    ┆ 0    ┆ 161 ┆ 1208                │
│ Commuting   ┆ 38   ┆ 166  ┆ 83   ┆ … ┆ 104  ┆ 41   ┆ 45  ┆ 932                 │
│ …           ┆ …    ┆ …    ┆ …    ┆ … ┆ …    ┆ …    ┆ …   ┆ …                   │
│ Stationery  ┆ 0    ┆ 38   ┆ 0    ┆ … ┆ 0    ┆ 0    ┆ 0   ┆ 38        